<a href="https://colab.research.google.com/github/LoosonWu/geek_homework/blob/main/homework_chap02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [11]:
%%cython

def hello():
    print("hello")

hello()

hello


In [10]:
%%cython
import numpy as np
cimport numpy as np


In [106]:
%%cython
import numpy as np
cimport numpy as np
import time as time
import pandas as pd

cpdef target_mean_v3(data, y_name, x_name):
    cdef long nrow = data.shape[0]
    cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrow), dtype=np.float64)
    cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float64)
    cdef np.ndarray[double] x = np.asfortranarray(data[x_name], dtype=np.float64)

    target_mean_v3_impl(result, y, x, nrow)
    return result

cdef void target_mean_v3_impl(double[:] result, double[:] y, double[:] x, const long nrow):
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()

    cdef long i
    for i in range(nrow):
        if x[i] not in value_dict.keys():
            value_dict[x[i]] = y[i]
            count_dict[x[i]] = 1
        else:
            value_dict[x[i]] += y[i]
            count_dict[x[i]] += 1
    i=0
    for i in range(nrow):
        result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)



In [23]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [124]:
%%cython
import numpy as np
cimport numpy as np
import time as time
import pandas as pd

cpdef target_mean_v4(data, y_name, x_name):
    cdef long nrow = data.shape[0]
    cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrow), dtype=np.float64)
    cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float64)
    cdef np.ndarray[double] x = np.asfortranarray(data[x_name], dtype=np.float64)

    target_mean_v4_impl(result, y, x, nrow)
    return result

cdef void target_mean_v4_impl(double[:] result, double[:] y, double[:] x, const long nrow):
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()
    cdef long i
    for i in range(10):
      value_dict[i] = 0
      count_dict[i] = 0

    i=0
    for i in range(nrow):
      value_dict[x[i]] += y[i]
      count_dict[x[i]] += 1
    i=0
    for i in range(nrow):
      result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)

In [125]:
y = np.random.randint(2, size=(5000, 1))

x = np.random.randint(10, size=(5000, 1))

data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
start = time.time()
result_2 = target_mean_v2(data, 'y', 'x')
end = time.time()
print(end - start)
start = time.time()
result_3 = target_mean_v3(data, 'y', 'x')
end = time.time()
print(end - start)
diff = np.linalg.norm(result_2 - result_3)
print(diff)
start = time.time()
result_4 = target_mean_v4(data, 'y', 'x')
end = time.time()
print(end - start)
diff4 = np.linalg.norm(result_2 - result_4)
print(diff4)

0.48299336433410645
0.0035452842712402344
0.0
0.0035216808319091797
0.0
